In [1]:
import csv
import sqlite3
import gzip
import json
import numpy as np
import pandas as pd

def db_check(name):
    conn = sqlite3.connect(name)
    c = conn.cursor()

    query = "SELECT name, sql FROM sqlite_master WHERE type='table'"
    c.execute(query)
    result = c.fetchall()

    for table_name, schema in result:
        print("---")
        row_count_query = f"SELECT COUNT(*) FROM {table_name}"
        c.execute(row_count_query)
        row_count = c.fetchone()[0]
        print(f"Table '{table_name}': {row_count} rows")
        print(f"Schema:\n{schema}\n")

    conn.close()

def db_clean(name):
    conn = sqlite3.connect(name)
    c = conn.cursor()
    c.execute("VACUUM")
    conn.close()

In [2]:
import time

conn = sqlite3.connect('project.db')
c = conn.cursor()

json_data = open('goodreads_book_authors.json')
table_name = "Authors"
drop_table_query = f"DROP TABLE IF EXISTS {table_name}"
c.execute(drop_table_query)

column_names = ['author_id', 'name']

create_table_query = f'''CREATE TABLE {table_name} (
                                        author_id text NOT NULL PRIMARY KEY,
                                        name text NOT NULL)'''
c.execute(create_table_query)

s = time.time()
count = 0
for row in json_data:
    row = json.loads(row)
    keys = [row[c] for c in column_names]

    insert_query = f"INSERT INTO {table_name} VALUES ({','.join(['?'] * len(keys))})"
    c.execute(insert_query, keys)
    count += 1
    if count%100000 == 0:
        e = time.time()
        print(count, e-s)

conn.commit()
conn.close()
db_check('project.db')

100000 0.26842403411865234
200000 0.5922689437866211
300000 0.9896059036254883
400000 1.4068329334259033
500000 1.8224880695343018
600000 2.252362012863159
700000 2.7046070098876953
800000 3.129926919937134
---
Table 'Authors': 829529 rows
Schema:
CREATE TABLE Authors (
                                        author_id text NOT NULL PRIMARY KEY,
                                        name text NOT NULL)



In [3]:
import time

conn = sqlite3.connect('project.db')
c = conn.cursor()

json_data = open('goodreads_books.json')
table_name = "Books"
drop_table_query = f"DROP TABLE IF EXISTS {table_name}"
c.execute(drop_table_query)

column_names = ['book_id', 'title', 'authors', 'image_url', 'description', 'average_rating']
create_table_query = f'''CREATE TABLE {table_name} (
                                        book_id text NOT NULL PRIMARY KEY,
                                        title text NOT NULL,
                                        author_id text NOT NULL,
                                        image_url text NOT NULL,
                                        description text NOT NULL,
                                        average_rating real NOT NULL,
                                        FOREIGN KEY (author_id) REFERENCES Authors(author_id))'''
c.execute(create_table_query)

s = time.time()
count = 0
for row in json_data:
    row = json.loads(row)

    keys = [row[c] for c in column_names]
    if not len(keys[0]) or not len(keys[1]) or not len(keys[2]) or not len(keys[3]) or not len(keys[4]) or not len(keys[5]) or row['language_code']!='eng':
        continue
    
    keys[2] = keys[2][0]['author_id']

    insert_query = f"INSERT INTO {table_name} VALUES ({','.join(['?'] * len(keys))})"
    c.execute(insert_query, keys)
    count += 1
    if count%100000 == 0:
        e = time.time()
        print(count, e-s)

conn.commit()
conn.close()

e = time.time()
print(count, e-s)
db_check('project.db')

100000 6.799588203430176
200000 13.732880115509033
300000 20.84645915031433
400000 28.07469415664673
500000 35.1120491027832
600000 42.58813524246216
645992 45.97576117515564
---
Table 'Authors': 829529 rows
Schema:
CREATE TABLE Authors (
                                        author_id text NOT NULL PRIMARY KEY,
                                        name text NOT NULL)

---
Table 'Books': 645992 rows
Schema:
CREATE TABLE Books (
                                        book_id text NOT NULL PRIMARY KEY,
                                        title text NOT NULL,
                                        author_id text NOT NULL,
                                        image_url text NOT NULL,
                                        description text NOT NULL,
                                        average_rating real NOT NULL,
                                        FOREIGN KEY (author_id) REFERENCES Authors(author_id))



In [4]:
conn = sqlite3.connect("project.db")
c = conn.cursor()

delete_query = """
DELETE FROM Authors
WHERE author_id NOT IN (
    SELECT author_id
    FROM Books
)
"""
c.execute(delete_query)

conn.commit()
conn.close()
db_check('project.db')

---
Table 'Authors': 179347 rows
Schema:
CREATE TABLE Authors (
                                        author_id text NOT NULL PRIMARY KEY,
                                        name text NOT NULL)

---
Table 'Books': 645992 rows
Schema:
CREATE TABLE Books (
                                        book_id text NOT NULL PRIMARY KEY,
                                        title text NOT NULL,
                                        author_id text NOT NULL,
                                        image_url text NOT NULL,
                                        description text NOT NULL,
                                        average_rating real NOT NULL,
                                        FOREIGN KEY (author_id) REFERENCES Authors(author_id))



In [5]:
import time

conn = sqlite3.connect('project.db')
c = conn.cursor()

json_data = open('goodreads_reviews_dedup.json')
table_name = "Reviews"
drop_table_query = f"DROP TABLE IF EXISTS {table_name}"
c.execute(drop_table_query)

column_names = ['user_id', 'book_id', 'rating', 'review_text']
create_table_query = f'''CREATE TABLE {table_name} (
                                        user_id text NOT NULL,
                                        book_id text NOT NULL,
                                        rating INTEGER NOT NULL,
                                        review_text text NOT NULL)'''
c.execute(create_table_query)

s = time.time()
count = 0
for row in json_data:
    row = json.loads(row)

    keys = [row[c] for c in column_names]

    if not len(keys[0]) or not len(keys[1]) or not len(keys[3]):
        continue

    insert_query = f"INSERT INTO {table_name} VALUES ({','.join(['?'] * len(keys))})"
    c.execute(insert_query, keys)
    count += 1
    if count%1000000 == 0:
        e = time.time()
        print(count, e-s)

conn.commit()
conn.close()

e = time.time()
print(count, e-s)
db_check('project.db')

1000000 5.322841167449951
2000000 10.517804145812988
3000000 18.094550132751465
4000000 25.998149156570435
5000000 31.426308155059814
6000000 37.65820026397705
7000000 47.296489238739014
8000000 55.45289611816406
9000000 60.784510374069214
10000000 66.95012831687927
11000000 76.74136233329773
12000000 85.18802833557129
13000000 90.47358536720276
14000000 96.18811225891113
15000000 105.35374426841736
15733029 111.47478914260864
---
Table 'Authors': 179347 rows
Schema:
CREATE TABLE Authors (
                                        author_id text NOT NULL PRIMARY KEY,
                                        name text NOT NULL)

---
Table 'Books': 645992 rows
Schema:
CREATE TABLE Books (
                                        book_id text NOT NULL PRIMARY KEY,
                                        title text NOT NULL,
                                        author_id text NOT NULL,
                                        image_url text NOT NULL,
                                        d

In [6]:
conn = sqlite3.connect("project.db")
c = conn.cursor()

delete_query = """
DELETE FROM Reviews
WHERE book_id NOT IN (
    SELECT book_id
    FROM Books
)
"""
c.execute(delete_query)

conn.commit()
conn.close()
db_check('project.db')

---
Table 'Authors': 179347 rows
Schema:
CREATE TABLE Authors (
                                        author_id text NOT NULL PRIMARY KEY,
                                        name text NOT NULL)

---
Table 'Books': 645992 rows
Schema:
CREATE TABLE Books (
                                        book_id text NOT NULL PRIMARY KEY,
                                        title text NOT NULL,
                                        author_id text NOT NULL,
                                        image_url text NOT NULL,
                                        description text NOT NULL,
                                        average_rating real NOT NULL,
                                        FOREIGN KEY (author_id) REFERENCES Authors(author_id))

---
Table 'Reviews': 8991291 rows
Schema:
CREATE TABLE Reviews (
                                        user_id text NOT NULL,
                                        book_id text NOT NULL,
                                        rating I

In [7]:
conn = sqlite3.connect("project.db")
cursor = conn.cursor()

query = "SELECT COUNT(DISTINCT user_id) FROM Reviews"
cursor.execute(query)

result = cursor.fetchone()[0]
print("Number of unique users:", result)

conn.close()

Number of unique users: 399911


In [8]:
conn = sqlite3.connect('project.db')
cursor = conn.cursor()

cursor.execute("""
    SELECT user_id, COUNT(*) as count
    FROM Reviews
    GROUP BY user_id
    ORDER BY count ASC
    LIMIT 389911
""")
result = cursor.fetchall()

cursor.execute("CREATE TEMPORARY TABLE temp_table (user_id TEXT)")
for row in result:
    value = row[0]
    cursor.execute("INSERT INTO temp_table VALUES (?)", (value,))

cursor.execute("""
    DELETE FROM Reviews
    WHERE user_id IN (
        SELECT user_id
        FROM temp_table
    )
""")

conn.commit()
conn.close()
db_check('project.db')

---
Table 'Authors': 179347 rows
Schema:
CREATE TABLE Authors (
                                        author_id text NOT NULL PRIMARY KEY,
                                        name text NOT NULL)

---
Table 'Books': 645992 rows
Schema:
CREATE TABLE Books (
                                        book_id text NOT NULL PRIMARY KEY,
                                        title text NOT NULL,
                                        author_id text NOT NULL,
                                        image_url text NOT NULL,
                                        description text NOT NULL,
                                        average_rating real NOT NULL,
                                        FOREIGN KEY (author_id) REFERENCES Authors(author_id))

---
Table 'Reviews': 3374600 rows
Schema:
CREATE TABLE Reviews (
                                        user_id text NOT NULL,
                                        book_id text NOT NULL,
                                        rating I

In [9]:
conn = sqlite3.connect("project.db")
cursor = conn.cursor()

query = "SELECT COUNT(DISTINCT user_id) FROM Reviews"
cursor.execute(query)

result = cursor.fetchone()[0]
print("Number of unique users:", result)

conn.close()

Number of unique users: 10000


In [10]:
conn = sqlite3.connect("project.db")
c = conn.cursor()

delete_query = """
DELETE FROM Books
WHERE book_id NOT IN (
    SELECT book_id
    FROM Reviews
)
"""
c.execute(delete_query)

conn.commit()
conn.close()
db_check('project.db')

---
Table 'Authors': 179347 rows
Schema:
CREATE TABLE Authors (
                                        author_id text NOT NULL PRIMARY KEY,
                                        name text NOT NULL)

---
Table 'Books': 427314 rows
Schema:
CREATE TABLE Books (
                                        book_id text NOT NULL PRIMARY KEY,
                                        title text NOT NULL,
                                        author_id text NOT NULL,
                                        image_url text NOT NULL,
                                        description text NOT NULL,
                                        average_rating real NOT NULL,
                                        FOREIGN KEY (author_id) REFERENCES Authors(author_id))

---
Table 'Reviews': 3374600 rows
Schema:
CREATE TABLE Reviews (
                                        user_id text NOT NULL,
                                        book_id text NOT NULL,
                                        rating I

In [11]:
conn = sqlite3.connect("project.db")
c = conn.cursor()

delete_query = """
DELETE FROM Authors
WHERE author_id NOT IN (
    SELECT author_id
    FROM Books
)
"""
c.execute(delete_query)

conn.commit()
conn.close()
db_check('project.db')

---
Table 'Authors': 107372 rows
Schema:
CREATE TABLE Authors (
                                        author_id text NOT NULL PRIMARY KEY,
                                        name text NOT NULL)

---
Table 'Books': 427314 rows
Schema:
CREATE TABLE Books (
                                        book_id text NOT NULL PRIMARY KEY,
                                        title text NOT NULL,
                                        author_id text NOT NULL,
                                        image_url text NOT NULL,
                                        description text NOT NULL,
                                        average_rating real NOT NULL,
                                        FOREIGN KEY (author_id) REFERENCES Authors(author_id))

---
Table 'Reviews': 3374600 rows
Schema:
CREATE TABLE Reviews (
                                        user_id text NOT NULL,
                                        book_id text NOT NULL,
                                        rating I

In [12]:
conn = sqlite3.connect('project.db')
c = conn.cursor()

create_table_query = f'''CREATE TABLE Users (
                                        user_id text NOT NULL PRIMARY KEY,
                                        id text,
                                        pw text,
                                        nickname text,
                                        email text,
                                        address text,
                                        address_gu text,
                                        address_dong text,
                                        lat_long text,
                                        lat real,
                                        long real
                                        )'''
c.execute(create_table_query)
c.execute('INSERT INTO Users (user_id) SELECT DISTINCT user_id FROM Reviews')

conn.commit()
conn.close()
db_check('project.db')

---
Table 'Authors': 107372 rows
Schema:
CREATE TABLE Authors (
                                        author_id text NOT NULL PRIMARY KEY,
                                        name text NOT NULL)

---
Table 'Books': 427314 rows
Schema:
CREATE TABLE Books (
                                        book_id text NOT NULL PRIMARY KEY,
                                        title text NOT NULL,
                                        author_id text NOT NULL,
                                        image_url text NOT NULL,
                                        description text NOT NULL,
                                        average_rating real NOT NULL,
                                        FOREIGN KEY (author_id) REFERENCES Authors(author_id))

---
Table 'Reviews': 3374600 rows
Schema:
CREATE TABLE Reviews (
                                        user_id text NOT NULL,
                                        book_id text NOT NULL,
                                        rating I

In [13]:
conn = sqlite3.connect('project.db')

cursor = conn.cursor()
cursor.execute("ALTER TABLE Reviews ADD COLUMN review_id text")

cursor.execute("SELECT rowid, * FROM Reviews")
rows = cursor.fetchall()

for index, row in enumerate(rows, start=1):
    row_id = row[0] 
    cursor.execute("UPDATE Reviews SET review_id = ? WHERE rowid = ?", (index, row_id))

conn.commit()
conn.close()
db_check('project.db')

---
Table 'Authors': 107372 rows
Schema:
CREATE TABLE Authors (
                                        author_id text NOT NULL PRIMARY KEY,
                                        name text NOT NULL)

---
Table 'Books': 427314 rows
Schema:
CREATE TABLE Books (
                                        book_id text NOT NULL PRIMARY KEY,
                                        title text NOT NULL,
                                        author_id text NOT NULL,
                                        image_url text NOT NULL,
                                        description text NOT NULL,
                                        average_rating real NOT NULL,
                                        FOREIGN KEY (author_id) REFERENCES Authors(author_id))

---
Table 'Reviews': 3374600 rows
Schema:
CREATE TABLE Reviews (
                                        user_id text NOT NULL,
                                        book_id text NOT NULL,
                                        rating I

In [14]:
conn = sqlite3.connect('project.db')
cursor = conn.cursor()

cursor.execute('''CREATE TABLE Reviews_temp (review_id text NOT NULL PRIMARY KEY,
                                        user_id text NOT NULL,
                                        book_id text NOT NULL,
                                        rating INTEGER NOT NULL,
                                        review_text text NOT NULL,
                                        FOREIGN KEY (user_id) REFERENCES Users(user_id),
                                        FOREIGN KEY (book_id) REFERENCES Books(book_id))''')
cursor.execute('INSERT INTO Reviews_temp SELECT review_id, user_id, book_id, rating, review_text FROM Reviews')
cursor.execute('DROP TABLE Reviews')
cursor.execute('ALTER TABLE Reviews_temp RENAME TO Reviews')


conn.commit()
cursor.close()
conn.close()
db_check('project.db')

---
Table 'Authors': 107372 rows
Schema:
CREATE TABLE Authors (
                                        author_id text NOT NULL PRIMARY KEY,
                                        name text NOT NULL)

---
Table 'Books': 427314 rows
Schema:
CREATE TABLE Books (
                                        book_id text NOT NULL PRIMARY KEY,
                                        title text NOT NULL,
                                        author_id text NOT NULL,
                                        image_url text NOT NULL,
                                        description text NOT NULL,
                                        average_rating real NOT NULL,
                                        FOREIGN KEY (author_id) REFERENCES Authors(author_id))

---
Table 'Users': 10000 rows
Schema:
CREATE TABLE Users (
                                        user_id text NOT NULL PRIMARY KEY,
                                        id text,
                                        pw text,
       

In [15]:
db_clean('project.db')

In [16]:
conn = sqlite3.connect('project.db')
cursor = conn.cursor()

query = "SELECT * FROM Reviews LIMIT 100"
cursor.execute(query)

rows = cursor.fetchall()
for row in rows:
    print(row)

conn.close()

('1', '01ec1a320ffded6b2dd47833f2c8e4fb', '34426579', 5, '4.5 I\'m-Obsessed Stars!! \n Jessica Hawkins has written an engrossing and emotional romance that I simply couldn\'t put down! I am obsessed with Lake and Manning. Book #3 is releasing on 10/23 so if you\'ve been waiting to start this series, don\'t delay! \n "Ten months. That was how long I\'d gone without a fix. Lake\'s letters came every couple weeks. I\'d opened the first one, but that was it. I\'d known right then that if I was going to make it through this, I couldn\'t think of her. Couldn\'t be inside her head that way, and she definitely couldn\'t be inside mine." \n My heart broke for Lake and Manning. They endured years apart and obstacles at every turn. I honestly do not know what will happen, but I can\'t wait to find out! \n "But he was still the one who could get everything I wanted with one word from Lake." \n This is the kind of book that I read every chance I could. The characters are all believable and very rea

In [17]:
'''
---------------------------------------------------------------
I worked on csv files but figured out we do not really need it
---------------------------------------------------------------

conn = sqlite3.connect("project.db")
c = conn.cursor()
table_name = "book_id_map"
drop_table_query = f"DROP TABLE IF EXISTS {table_name}"
c.execute(drop_table_query)

create_table = f'CREATE TABLE {table_name}(
                book_id_csv text NOT NULL PRIMARY KEY,
                book_id text NOT NULL);
                '
c.execute(create_table)

column_names = ['book_id_csv', 'book_id']
with open("book_id_map.csv", "r") as file:
    csv_reader = csv.reader(file)
    next(csv_reader)  

    for row in csv_reader:

        insert_query = f"""
        INSERT INTO {table_name} ({','.join(col for col in column_names)}) VALUES ({','.join(['?'] * len(column_names))})
        """
        c.execute(insert_query, row)
conn.commit()
conn.close()
db_check('project.db')

conn = sqlite3.connect("project.db")
c = conn.cursor()
table_name = "user_id_map"
drop_table_query = f"DROP TABLE IF EXISTS {table_name}"
c.execute(drop_table_query)

create_table = f'CREATE TABLE {table_name}(
                user_id_csv text NOT NULL PRIMARY KEY,
                user_id text NOT NULL);
                '
c.execute(create_table)

column_names = ['user_id_csv', 'user_id']
with open("user_id_map.csv", "r") as file:
    csv_reader = csv.reader(file)
    next(csv_reader)  

    for row in csv_reader:

        insert_query = f"""
        INSERT INTO {table_name} ({','.join(col for col in column_names)}) VALUES ({','.join(['?'] * len(column_names))})
        """
        c.execute(insert_query, row)
conn.commit()
conn.close()
db_check('project.db')


conn = sqlite3.connect("project.db")
c = conn.cursor()
table_name = "Interactions"
drop_table_query = f"DROP TABLE IF EXISTS {table_name}"
c.execute(drop_table_query)

create_table = f'CREATE TABLE {table_name}(
                user_id_csv text NOT NULL,
                book_id_csv text NOT NULL,
                is_read INTEGER NOT NULL,
                rating real NOT NULL,
                is_reviewed INTEGER NOT NULL
                );
                '

c.execute(create_table)

column_names = ['user_id_csv', 'book_id_csv', 'is_read', 'rating', 'is_reviewed']

with open("goodreads_interactions.csv", "r") as file:
    csv_reader = csv.reader(file)
    next(csv_reader)
    s = time.time()
    count = 0  
    for row in csv_reader:
        if row[2] == '0':
            continue
        insert_query = f"""
        INSERT INTO {table_name} ({','.join(col for col in column_names)}) VALUES ({','.join(['?'] * len(column_names))})
        """
        c.execute(insert_query, row)

        count += 1
        if count%10000000 == 0:
            e = time.time()
            print(count, e-s)
conn.commit()
conn.close()
db_check('project.db')
'''

'\n---------------------------------------------------------------\nI worked on csv files but figured out we do not really need it\n---------------------------------------------------------------\n\nconn = sqlite3.connect("project.db")\nc = conn.cursor()\ntable_name = "book_id_map"\ndrop_table_query = f"DROP TABLE IF EXISTS {table_name}"\nc.execute(drop_table_query)\n\ncreate_table = f\'CREATE TABLE {table_name}(\n                book_id_csv text NOT NULL PRIMARY KEY,\n                book_id text NOT NULL);\n                \'\nc.execute(create_table)\n\ncolumn_names = [\'book_id_csv\', \'book_id\']\nwith open("book_id_map.csv", "r") as file:\n    csv_reader = csv.reader(file)\n    next(csv_reader)  \n\n    for row in csv_reader:\n\n        insert_query = f"""\n        INSERT INTO {table_name} ({\',\'.join(col for col in column_names)}) VALUES ({\',\'.join([\'?\'] * len(column_names))})\n        """\n        c.execute(insert_query, row)\nconn.commit()\nconn.close()\ndb_check(\'project